# Home task: pandas 

## Question 1

- Load the energy data from the file [Energy Indicators.xls](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls).
It is a list of indicators of energy supply and renewable electricity production from the United Nations for the year 2013.


- It should be put into a DataFrame with the variable name of "energy"


- Make sure to exclude the footer and header information from the datafile.


- The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:<br>
`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`


- Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule).


- For all countries which have missing data (e.g. data with `...`) make sure this is reflected as `np.NaN` values.


- Rename the following list of countries (for use in later questions):
    - `Republic of Korea`: `South Korea`,
    - `United States of America`: `United States`,
    - `United Kingdom of Great Britain and Northern Ireland`: `United Kingdom`,
    - `China, Hong Kong Special Administrative Region`: `Hong Kong`


- There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g.:
    - `Bolivia (Plurinational State of)` should be `Bolivia`,
    - `Switzerland17` should be `Switzerland`.


- Next, load the GDP data from the file ["world_bank.csv"](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). 
It is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame "GDP"


- Make sure to skip the header, and rename the following list of countries:
    - `Korea, Rep.`: `South Korea`,
    - `Iran, Islamic Rep.`: `Iran`,
    - `Hong Kong SAR, China`: `Hong Kong`


- Finally, load the "Sciamgo Journal and Country Rank data for [Energy Engineering and Power Technology"](http://www.scimagojr.com/countryrank.php?category=2102). It ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame "ScimEn"


- Join the three datasets: Energy, GDP, and ScimEn into a new dataset (using the intersection of country names). Use only the 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).


- The index of this DataFrame should be the name of the country, and the columns should be<br>
`['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', 2012', '2013', '2014', '2015']`

Function "answer_one" should return the resulted DataFrame (20 columns and 15 entries)

In [10]:
import pandas as pd
import numpy as np

energy = pd.read_excel('../data/Energy Indicators.xls', skiprows=17, skipfooter=38)
energy.drop(columns=energy.columns[:2], inplace=True)
energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Country                   227 non-null    object 
 1   Energy Supply             227 non-null    object 
 2   Energy Supply per Capita  227 non-null    object 
 3   % Renewable               227 non-null    float64
dtypes: float64(1), object(3)
memory usage: 7.2+ KB


In [11]:
# dataframe has no obvious missing values, but..
try:
    energy['Energy Supply'] = energy['Energy Supply'].astype('float64')
except ValueError as e:
    print(e)

could not convert string to float: '...'


In [12]:
energy.replace('...', np.nan, inplace=True) # could also be done at the beginning with read_excel using na_values=['...']
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Country                   227 non-null    object 
 1   Energy Supply             222 non-null    float64
 2   Energy Supply per Capita  222 non-null    float64
 3   % Renewable               227 non-null    float64
dtypes: float64(3), object(1)
memory usage: 7.2+ KB


In [13]:
# dtype changed to float64, so there is no more missing values to mark
energy['Energy Supply'] = energy['Energy Supply'] * 1000000
energy.head()

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,3.210000e+08,10.0,78.669280
1,Albania,1.020000e+08,35.0,100.000000
2,Algeria,1.959000e+09,51.0,0.551010
3,American Samoa,NaN,NaN,0.641026
4,Andorra,9.000000e+06,121.0,88.695650


In [14]:
# display countries with numbers or parenthesis in their names
dirty_country_names = energy['Country'][energy['Country'].str.contains('\d|\(|\)')]
print(dirty_country_names)
print(dirty_country_names[214])

11                                            Australia1
24                      Bolivia (Plurinational State of)
42                                                China2
43       China, Hong Kong Special Administrative Region3
44           China, Macao Special Administrative Region4
58                                              Denmark5
70                           Falkland Islands (Malvinas)
73                                               France6
83                                            Greenland7
97                                            Indonesia8
98                            Iran (Islamic Republic of)
103                                               Italy9
105                                              Japan10
111                                             Kuwait11
133                     Micronesia (Federated States of)
143                                        Netherlands12
161                                           Portugal13
177                            

In [15]:
# so parenthesis and numbers are appended to the ends of country names
# in a particular way, so we can use regex to clean them up
energy['Country'] = energy['Country'].str.replace(r'\d+', '', regex=True)
energy['Country'] = energy['Country'].str.replace(r'\s+\(.*\)', '', regex=True)

energy['Country'].replace({
    'Republic of Korea': 'South Korea',
    'United States of America': 'United States',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'China, Hong Kong Special Administrative Region': 'Hong Kong'
}, inplace=True)

energy['Country'][dirty_country_names.index]

11                                      Australia
24                                        Bolivia
42                                          China
43                                      Hong Kong
44     China, Macao Special Administrative Region
58                                        Denmark
70                               Falkland Islands
73                                         France
83                                      Greenland
97                                      Indonesia
98                                           Iran
103                                         Italy
105                                         Japan
111                                        Kuwait
133                                    Micronesia
143                                   Netherlands
161                                      Portugal
177                                  Saudi Arabia
179                                        Serbia
183                                  Sint Maarten


In [16]:
# unlike in the assignment, I'm using file with more recent data up to 2022,
# assuming it's acceptable
GDP = pd.read_csv('../data/world_bank.csv', skiprows=4)
GDP['Country Name'].replace({
    'Korea, Rep.': 'South Korea',
    'Iran, Islamic Rep.': 'Iran',
    'Hong Kong SAR, China': 'Hong Kong'
}, inplace=True)
GDP.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.791061e+09,2.963128e+09,2.983799e+09,3.092179e+09,3.276188e+09,3.395794e+09,2.610039e+09,3.126019e+09,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.112502e+10,2.161623e+10,2.350628e+10,2.804836e+10,2.592067e+10,2.947210e+10,...,1.006526e+12,9.273485e+11,8.851764e+11,1.021043e+12,1.007196e+12,1.000834e+12,9.275933e+11,1.081998e+12,1.169484e+12,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,2.055058e+10,1.999814e+10,1.801955e+10,1.889635e+10,1.841886e+10,1.890450e+10,2.014345e+10,1.458314e+10,NaN,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.044764e+10,1.117321e+10,1.199053e+10,1.272769e+10,1.389811e+10,1.492979e+10,...,8.943225e+11,7.686447e+11,6.913634e+11,6.848988e+11,7.670257e+11,8.225384e+11,7.864600e+11,8.444597e+11,8.778633e+11,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.372444e+11,8.721930e+10,4.984049e+10,6.897277e+10,7.779294e+10,6.930911e+10,5.024137e+10,6.568544e+10,1.067136e+11,NaN


In [17]:
ScimEn = pd.read_excel('../data/scimagojr.xlsx')
ScimEn.head(15)

,Rank,Country,Region,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,Asiatic Region,360468,358777,3947871,2705774,10.95,308
1,2,United States,Northern America,199442,195042,3068926,881789,15.39,422
2,3,India,Asiatic Region,76103,74167,760964,280893,10.00,217
3,4,Japan,Asiatic Region,56249,55680,633294,136132,11.26,217
4,5,United Kingdom,Western Europe,52572,51156,909276,151672,17.30,267
5,6,Germany,Western Europe,47781,46767,641717,133693,13.43,230
6,7,Russian Federation,Eastern Europe,43567,43290,175721,79765,4.03,103
7,8,Canada,Northern America,39036,38276,787010,125333,20.16,263
8,9,Italy,Western Europe,35991,34424,529459,123042,14.71,192
9,10,South Korea,Asiatic Region,35294,35005,503147,87529,14.26,182


In [18]:
# ScimEn seems to be already sorted by Rank
# from ScimEn we need all columns except Region
joined_df = ScimEn.loc[:14, ScimEn.columns != 'Region'].join(
    energy.set_index('Country'),
    on='Country'
).join(
    GDP.set_index('Country Name').loc[:, '2006':'2015'],
    on='Country'
).set_index('Country')
joined_df

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,360468,358777,3947871,2705774,10.95,308,1.271910e+11,93.0,19.754910,2.752119e+12,3.550328e+12,4.594337e+12,5.101691e+12,6.087192e+12,7.551545e+12,8.532186e+12,9.570471e+12,1.047562e+13,1.106157e+13
United States,2,199442,195042,3068926,881789,15.39,422,9.083800e+10,286.0,11.570980,1.381559e+13,1.447423e+13,1.476986e+13,1.447806e+13,1.504896e+13,1.559973e+13,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13
India,3,76103,74167,760964,280893,10.00,217,3.319500e+10,26.0,14.969080,9.402599e+11,1.216736e+12,1.198895e+12,1.341888e+12,1.675616e+12,1.823052e+12,1.827638e+12,1.856721e+12,2.039126e+12,2.103588e+12
Japan,4,56249,55680,633294,136132,11.26,217,1.898400e+10,149.0,10.232820,4.601663e+12,4.579750e+12,5.106679e+12,5.289494e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
United Kingdom,5,52572,51156,909276,151672,17.30,267,7.920000e+09,124.0,10.600470,2.709978e+12,3.092996e+12,2.931684e+12,2.417566e+12,2.491397e+12,2.666403e+12,2.706341e+12,2.786315e+12,3.065223e+12,2.934858e+12
Germany,6,47781,46767,641717,133693,13.43,230,1.326100e+10,165.0,17.901530,2.994704e+12,3.425578e+12,3.745264e+12,3.411261e+12,3.399668e+12,3.749315e+12,3.527143e+12,3.733805e+12,3.889093e+12,3.357586e+12
Russian Federation,7,43567,43290,175721,79765,4.03,103,3.070900e+10,214.0,17.288680,9.899321e+11,1.299703e+12,1.660848e+12,1.222646e+12,1.524917e+12,2.045923e+12,2.208294e+12,2.292470e+12,2.059242e+12,1.363482e+12
Canada,8,39036,38276,787010,125333,20.16,263,1.043100e+10,296.0,61.945430,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,35991,34424,529459,123042,14.71,192,6.530000e+09,109.0,33.667230,1.949552e+12,2.213102e+12,2.408655e+12,2.199929e+12,2.136100e+12,2.294994e+12,2.086958e+12,2.141924e+12,2.162010e+12,1.836638e+12


In [19]:
def answer_one():
    return joined_df # should be a view, not a copy

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 2
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [20]:
# I assume "last 10 years" here means '2006':'2015'
def answer_two():
    Top15 = answer_one()
    average_sr = Top15.loc[:, '2006':'2015'].mean(axis=1)
    average_sr.name = 'avgGDP'
    average_sr.sort_values(ascending=False, inplace=True)
    return average_sr
    
answer_two()

Country
United States         1.570403e+13
China                 6.927707e+12
Japan                 5.239642e+12
Germany               3.523342e+12
United Kingdom        2.780276e+12
France                2.691337e+12
Italy                 2.142986e+12
Brazil                1.988889e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.400886e+12
South Korea           1.221372e+12
Australia             1.207513e+12
Iran                  4.563261e+11
Name: avgGDP, dtype: float64

### Question 3
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [21]:
def answer_three():
    Top15 = answer_one()
    average_6th_country = answer_two().index[5]
    return Top15.loc[average_6th_country, '2015'] - Top15.loc[average_6th_country, '2006']

answer_three()

118652421857.7998

### Question 4

Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [22]:
def answer_four():
    Top15 = answer_one()
    Top15['Citation ratio'] = Top15['Self-citations'] / Top15['Citations']
    index_max = Top15['Citation ratio'].idxmax()
    return index_max, Top15.loc[index_max, 'Citation ratio']

answer_four()

('China', 0.6853754846599598)

### Question 5

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [23]:
def answer_five():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    return Top15['Population'].sort_values(ascending=False).index[2]

answer_five()

'United States'

### Question 6
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*


In [24]:
def answer_six():
    Top15 = answer_one()
    Top15['Citable documents per person'] = Top15['Citable documents'] / Top15['Population']
    return Top15['Citable documents per person'].corr(Top15['Energy Supply per Capita'])

answer_six()

0.7114342527331208

### Question 7
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [25]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

def answer_seven():
    Top15 = answer_one()
    Top15['Continent'] = pd.Series(ContinentDict)
    return Top15.groupby('Continent')['Population'].agg(['size', 'sum', 'mean', 'std'])

answer_seven() 
# std is NaN because there's only one country in Australia and South America

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,6,4.579297e+08,7.632161e+07,3.464767e+07
North America,2,3.528552e+08,1.764276e+08,1.996696e+08
South America,1,2.059153e+08,2.059153e+08,NaN
